In [ ]:
#default_exp core

# rfpfolio

> Create portfolios with rebalancing, and measure performance.

In [ ]:
#hide

# Do this to see possible %nbdev_ magics
#from nbdev import *

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

## Saying Stuff

In [ ]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
say_hello("Rick")

'Hello Rick!'

In [ ]:
test_eq(say_hello("Ajay"), "Hello Ajay!")